# 0. Introdução

**Created by:** Pedro V. Pinho N.

**Date:** 05/03/2020

**Contact:** peu.vpn@gmail.com

Nesse notebook está registrado a minha primeira tentativa com as competições de Machine Learning do Kaggle. O projeto registado aqui é referente a competição **"Titanic: Machine Learning from Disaster"**, https://www.kaggle.com/c/titanic, é uma competição de nível iniciante focada em Análise Exploratória de Dados e Engenharia de Parâmetros com o objetivo de desenvolver modelos preditivos para prever quais passageiros irão sobreviver ao acidente, baseado em parâmetros como ***idade**, **sexo**, **classe social**, **quantidade de filhos a bordo**, etc.

Diversos outros kernels foram extensivamente consultados para auxiliar na contrução desse notebook. De todos, dois deles devem ser mencionados devido sua grande importância: **Titanic - Advanced Feature Engineering Tutorial** (https://www.kaggle.com/gunesevitan/titanic-advanced-feature-engineering-tutorial) e **Titanic: on the top with a simple model** (https://www.kaggle.com/goldens/titanic-on-the-top-with-a-simple-model). Muitas informações são tiradas da enciclopedia do Titanic (https://www.encyclopedia-titanica.org/). Este notebook é a minha forma de fazer proveitos da conquista e expertise de outros grandes Data Scientists, afim de que eu possa aperfeiçoar meus conhecimentos nessa crescente área de Análise de Dados.

## 0.1. Pacotes Utilizados

In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections as cl

## 0.2. Leitura dos dados de treinamento

In [120]:
# Concatena dois DataFrames
def concat_df(df1, df2):
    return pd.concat([df1,df2], sort = True).reset_index(drop = True)

# Separa em dois DataFrames - Treinamento e Teste
def unconcat_df(df, len_train):
    return df.loc[:len_train - 1], df.loc[len_train:].drop(["Survived"], axis = 1)

#Leitura dos arquivos de teste e treinamento
df_train = pd.read_csv("./Data/train.csv")
df_test  = pd.read_csv("./Data/test.csv")

#Número entradas em cada parte dos dados
len_train = len(df_train)
len_test  = len(df_test)

# Data frame contendo todos os valores test + train
# Será utilizado para fazer as correções de 'missing values' e depois gerará os dois grupos (train e test)
df_all = concat_df(df_train, df_test)

#Nome dos DataFrames
df_train.name = 'Grupo de Treinamento'
df_test.name = 'Grupo de Teste'
df_all.name = 'Grupo Completo'

print('Formato do grupo de treinamento = {}'.format(df_train.shape))
print('Formato do grupo de teste = {}\n'.format(df_test.shape))

Formato do grupo de treinamento = (891, 12)
Formato do grupo de teste = (418, 11)



In [121]:
df_train.info()
df_train.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
116,117,0,3,"Connors, Mr. Patrick",male,70.5,0,0,370369,7.7500,NaN,Q
97,98,1,1,"Greenfield, Mr. William Bertram",male,23.0,0,1,PC 17759,63.3583,D10 D12,C
250,251,0,3,"Reed, Mr. James George",male,NaN,0,0,362316,7.2500,NaN,S


In [122]:
df_test.info()
df_test.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
78,970,2,"Aldworth, Mr. Charles Augustus",male,30.0,0,0,248744,13.000,NaN,S
239,1131,1,"Douglas, Mrs. Walter Donald (Mahala Dutton)",female,48.0,1,0,PC 17761,106.425,C86,C
72,964,3,"Nieminen, Miss. Manta Josefina",female,29.0,0,0,3101297,7.925,NaN,S


# 1. Preprocessamento e manipulação

* `PassengerId` identificação do passageiro, irrelevante para a análise
* `Survived` variável que desejamos prever (**0** or **1**):
    - **1 = Sobreviveu**
    - **0 = Não sobreviveu**
* `Pclass` (Passenger Class) é o status socio-econômico do passageiro e é categorizado em entradas númericas com **3** valores únicos (**1**,  **2 **or **3**):
    - **1 = Primeira Classe**
    - **2 = Classe Executiva**
    - **3 = Classe Econômica**
* `Name`, `Sex` and `Age` são auto-explicativos
* `SibSp` é o número total de irmãos e conjugês do passageiro
* `Parch` é o número total de parentes ou filhos do passageiro
* `Ticket` é o número do ticket do passageiro
* `Fare` é o valor pago pelo passageiro
* `Cabin` é a cabine do passageiro
* `Embarked` é o porto por qual o passageiro embarcou e é categorizado por 3 valores únicos (**C**, **Q** or **S**):
    - **C = Cherbourg**
    - **Q = Queenstown**
    - **S = Southampton**

## 1.1. Categorização dos Títulos Honoríficos

O objetivo dessa subseção é categorizar os diferentes títulos presentes nos nomes dos passageiros, como por exemplo "Braund, **Mr.** Owen Harris" ou "Bowerman, **Miss.** Elsie Edith". Entretanto, como será possível ver, teremos muito títulos diferentes, o que dificulta qualquer tipo de análise e construção de modelo preditivo, assim, iremos mapear os títulos para 6 categorias mais abrangentes.

Podemos ver que todos os nomes são compostos e estruturados da seguinte forma: **sobrenome**(virgula)**título**(ponto)(espaço)**nome**. Basta portanto, primeiramente separar o nome em virgulas, o que nós entrega uma lista da seguinte forma: [**sobrenome**, **título**(ponto)(espaço)**nome**]. Pegando o segundo elemento e agora separando em ponto temos [**título**,(espaço)**nome**]. Basta então pegar agora o primeiro elemento e temos o nosso título.

In [123]:
# Cria uma nova coluna 'Title' com os títulos dos passageiros para uma consequente extração
df_all['Title'] = df_all.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
print(df_all['Title'].value_counts(),'\n')

# Mapeamento dos títulos
map_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}

# Aplicando nos dados
df_all['Title'] = df_all['Title'].map(map_titles)
print(df_all['Title'].value_counts())

Mr              757
Miss            260
Mrs             197
Master           61
Dr                8
Rev               8
Col               4
Mlle              2
Ms                2
Major             2
Sir               1
Mme               1
Lady              1
Dona              1
Jonkheer          1
Don               1
the Countess      1
Capt              1
Name: Title, dtype: int64 

Mr         757
Miss       262
Mrs        200
Master      61
Officer     23
Royalty      6
Name: Title, dtype: int64


## 1.2. Dados faltando

### 1.2.A. Idade

Claramente usaremos algum tipo de média das idades para preencher os valores faltantes, entretanto, pessoas de diferentes esferas possuem idades médias diferentes, por exemplo, homens possuem uma idade média maior que mulheres, e por ai vai. Portanto precisamos saber quais esferas - Classe, Sexo, Título - mais se correlaciona com os valores de idade.

Veremos que o parâmentro númerico que mais se relaciona com a idade é a classe social (Pclass) e vemos também que dentro das classes sociais a mulher sempre tem a menor idade média. Portanto vamos preencher os valores NaN com as médias respectivas de cada esfera.

In [124]:
# Correlação da idade com os parâmetros de valores númericos
corr = df_all.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)
print(corr[corr['Feature 1'] == 'Age'],'\n')

# Agrupamento por 'Pclass' e 'Sex'
print(df_all.groupby(['Pclass', 'Sex']).median()['Age'])

# Preenchendo os valores faltantes com suas respectivas médias
df_all['Age'] = df_all.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))

   Feature 1    Feature 2  Correlation Coefficient
6        Age          Age                 1.000000
9        Age       Pclass                 0.408106
17       Age        SibSp                 0.243699
22       Age         Fare                 0.178740
25       Age        Parch                 0.150917
29       Age     Survived                 0.077221
41       Age  PassengerId                 0.028814 

Pclass  Sex   
1       female    36.0
        male      42.0
2       female    28.0
        male      29.5
3       female    22.0
        male      25.0
Name: Age, dtype: float64


### 1.2.B. Porto embarcado

In [125]:
# Encontrando valores faltantes
print(df_all[df_all['Embarked'].isnull()])

      Age Cabin Embarked  Fare                                       Name  \
61   38.0   B28      NaN  80.0                        Icard, Miss. Amelie   
829  62.0   B28      NaN  80.0  Stone, Mrs. George Nelson (Martha Evelyn)   

     Parch  PassengerId  Pclass     Sex  SibSp  Survived  Ticket Title  
61       0           62       1  female      0       1.0  113572  Miss  
829      0          830       1  female      0       1.0  113572   Mrs  


Podemos ver que os únicos valoes que faltam na secção 'Embarked' são de duas mulheres de primeira classe que curiosamente estão na mesma cabine e possuem o mesmo ticket. Podemos então que elas se conhecem. Uma rápida busca no Google (https://www.encyclopedia-titanica.org/titanic-survivor/martha-evelyn-stone.html) nos fornece a informação que Martha Evelyn embarcou com sua empregada Amelie Icard no porto de **Southampton**.

In [126]:
# Preenchendo os valores faltantes
df_all["Embarked"] = df_all['Embarked'].fillna('S')

### 1.2.C. Preço da passagem

Podemos ver abaixo que o preço da passagem é bastante correlato com o tamanho da familia (**Parch** e **SibSp**) e a classe social (**Pclass**). Portanto faremos um procedimento similar ao da idade, agora agrupando por **Parch** **Pclass** e **SibSp**.

In [127]:
# Corelação entre Fare e outros parâmentros
print(corr[corr['Feature 1'] == 'Fare'])

# Calculando a tarifa média
df_all['Fare'] = df_all.groupby(['Pclass', 'SibSp', 'Parch']).Fare.apply(lambda x: x.fillna(x.median()))

   Feature 1    Feature 2  Correlation Coefficient
2       Fare         Fare                 1.000000
7       Fare       Pclass                 0.558629
16      Fare     Survived                 0.257307
20      Fare        Parch                 0.221539
21      Fare          Age                 0.178740
23      Fare        SibSp                 0.160238
40      Fare  PassengerId                 0.031428


### 1.2.D. Cabine

Com relação às cabines temos o mesmo problema dos títulos, muitas categorias. Teremos que fazer uma análise da composição de cada categoria para que possamos saber como agrupar-las. Analisando a enciclopedia titanica é possível descobrir que a letra presente na cabine representa o deck, com essa informação podemos criar uma nova coluna para armazena-la.

In [151]:
# Criação de uma coluna com os decks
df_all['Deck'] = df_all['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else 'M')

df_all_deck = df_all.groupby(['Deck', 
                              'Pclass']).count().drop(columns = ['Age','Cabin',
                                                               'Embarked','Fare',
                                                               'Parch','PassengerId',
                                                               'Sex', 'SibSp',
                                                               'Survived',
                                                               'Ticket',
                                                               'Title']).rename({'Name': 'Counts'}).transpose()

Deck     A   B   C   D      E         F     G   M            T
Pclass   1   1   1   1  2   1  2  3   2  3  3   1    2    3  1
Name    22  65  94  40  6  34  4  3  13  8  5  67  254  693  1
